In [2]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [3]:
d_1 = pd.read_csv("data1.csv")
d_2 = pd.read_csv("data2.csv")
d_3 = pd.read_csv("data3.csv")
d_4 = pd.read_csv("data4.csv")
d_5 = pd.read_csv("data5.csv")
d = d_1.append(d_2).append(d_3).append(d_4).append(d_5)

In [4]:
print len(d)

337331


In [5]:
train_labels = pd.read_csv("../data/train_v2.csv")

In [6]:
d = d.fillna(0)
train = train_labels.merge(d, on='file', how='left')

In [7]:
train = train.dropna()

In [8]:
print len(train)

337042


,file,sponsored,Unnamed: 0,bad_links_number,braces,brackets,len_meta_content,length,lines,median_scale,num_all_links,num_image_class,num_image_src,num_links,num_script_src,num_script_type,spaces,tabs,widgets,words
1,845185_raw_html.txt,0,64647,952,41,20,35,14347,150,1,6,0,1,0,5,6,828,0,0,605
2,3370996_raw_html.txt,0,43967,427,0,0,1,88,4,0,0,0,0,0,0,0,7,0,0,8
3,3370996_raw_html.txt,0,43967,427,0,0,1,88,4,0,0,0,0,0,0,0,7,0,0,8
5,3273416_raw_html.txt,0,41954,4009,124,49,18,104343,1993,50520,119,4,23,71,25,28,15435,155,32,7829
8,781426_raw_html.txt,0,63656,3730,245,64,52,83032,1952,0,90,0,7,77,6,27,20263,0,32,4624
9,781426_raw_html.txt,0,63655,3730,245,64,52,83032,1952,0,90,0,7,77,6,27,20263,0,32,4624
10,2825110_raw_html.txt,0,33756,2931,36,186,34,53581,1118,0,120,8,8,100,12,15,10001,0,13,3143
11,2825110_raw_html.txt,0,33756,2931,36,186,34,53581,1118,0,120,8,8,100,12,15,10001,0,13,3143
12,3536138_raw_html.txt,0,46711,524,1,0,3,7021,210,4900,26,0,9,17,0,0,2212,0,0,479
13,2691934_raw_html.txt,0,31297,3116,33,21,20,42501,951,31200,79,0,16,53,10,26,7229,115,2,3007


In [14]:
clf = RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=0)
columns = ["num_script_src", "widgets", "spaces", "tabs", "words", "braces",\
                                       "brackets", "len_meta_content", "bad_links_number",\
                                       "num_script_type", "median_scale", "num_image_class"]
et_score = cross_val_score(clf, train[columns], train.sponsored, cv = 3, scoring="roc_auc")
print et_score.mean()

0.930382807017


In [15]:
test_labels= pd.read_csv("../data/sampleSubmission_v2.csv")
test_data = pd.read_csv("test.csv")
test = test_labels.merge(test_data, on="file", how='left')

In [16]:
test = test.fillna(0)

In [17]:
clf = RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=0)
clf.fit(train[columns], train.sponsored)
submission = test[['file']].reset_index(drop=True)
submission['sponsored'] = clf.predict_proba(test[columns])[:, 1]
submission.to_csv('submission.csv', index=False)